In [ ]:
!pip install tqdm
!pip install scikit-learn

     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.6 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.6 kB ? eta -:--:--
     ------------------------- ------------ 41.0/60.6 kB 393.8 kB/s eta 0:00:01
     -------------------------------------- 60.6/60.6 kB 402.7 kB/s eta 0:00:00
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/11.0 MB 1.9 MB/s eta 0:00:06
   - -------------------------------------- 0.3/11.0 MB 3.5 MB/s eta 0:00:04
   - -------------------------------------- 0.5/11.0 MB 3.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/11.0 MB 4.2 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/11.0 MB 4.3 MB/s eta 0:00:03
   ---- ----------------------------

In [ ]:
# importing libraries
from PIL import Image
import torch
from torch import nn
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm import trange, tqdm
from torchvision import models, transforms
import torchvision.transforms.functional as TF


In [ ]:
# dataloader with features also output
import os
import torch
import torchvision.transforms as transforms
from PIL import Image
from torchvision.models import resnet50

class oct_data(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.labels = []
        self.max = 300
        self.features = []

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load ResNet model for feature extraction
        resnet = resnet50(pretrained=True)
        self.resnet = torch.nn.Sequential(*(list(resnet.children())[:-1])).to(self.device)

        # resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
        # resnet.trainable = False
        # x = GlobalAveragePooling2D()(resnet.output)
        # self.resnet = Model(inputs = resnet.input, outputs=x)
        # -----------

        classes = sorted(os.listdir(root_dir))

        for class_idx, folder in enumerate(classes):
            class_dir = os.path.join(root_dir, folder)
            for p in os.listdir(class_dir):
                path = os.path.join(class_dir, p)
                self.data.append(path)
                self.labels.append(class_idx)
                # self.max = max(len(os.listdir(path)), self.max)

        # Preprocess transforms for images
        self.preprocess = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        # Precompute and cache features during initialization
        self.compute_features()

    def compute_features(self):
        for data_path in self.data:
            features = []
            for path in os.listdir(data_path):
                image_path = os.path.join(data_path, path)
                img = Image.open(image_path)
                img = img.convert('RGB')
                img = self.preprocess(img)
                # print(img.shape)
                with torch.no_grad():
                    img = img.unsqueeze(0).to(self.device)  # Add batch dimension
                    feature = self.resnet(img)
                features.append(feature)
                # print(feature.shape)
            # Pad or truncate features to a fixed length
            # features = self.pad(features)
            self.features.append(features)

    def pad(self, features):
        max_length = self.max
        while len(features) < max_length:
            features.extend(features[::-1] + features[:])
        return features[:max_length]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = self.labels[idx]
        original_images = []

        for path in os.listdir(self.data[idx]):
            image_path = os.path.join(self.data[idx], path)
            img = Image.open(image_path)
            original_images.append(img)

        original_images = [transforms.ToTensor()(img) for img in original_images]


        original_images = self.pad(original_images)
        features = self.pad(self.features[idx])

        out = torch.stack(features)
        out = torch.squeeze(out, dim=-1)
        out = torch.squeeze(out, dim=-1) # removing the last 2 dimensions of the features

        return torch.stack(original_images), out, label


In [ ]:
def patchify(images, n_patches):
    b, n, c, h, w = images.shape

    assert h == w, "Patchify method is implemented for square images only"

    patches = torch.zeros(b, n, n_patches ** 2, h * w * c // n_patches ** 2)
    patch_size = h // n_patches

    for idx, data_instance in enumerate(images):
      for image_idx, image in enumerate(data_instance):
        for i in range(n_patches):
            for j in range(n_patches):
                patch = image[:, i * patch_size: (i + 1) * patch_size, j * patch_size: (j + 1) * patch_size]
                patches[idx, image_idx, i * n_patches + j] = patch.flatten()
    return patches

def get_positional_embeddings(sequence_length, d):
    result = torch.ones(sequence_length, d)
    for i in range(sequence_length):
        for j in range(d):
            result[i][j] = np.sin(i / (10000 ** (j / d))) if j % 2 == 0 else np.cos(i / (10000 ** ((j - 1) / d)))
    return result


In [ ]:

class SelfAttention(nn.Module):
  def __init__(self, d, n_heads=2):
    super(SelfAttention, self).__init__()
    self.d = d
    self.n_heads = n_heads

    assert d % n_heads == 0

    d_head = int(d / n_heads)

    self.q_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(n_heads)])
    self.k_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(n_heads)])
    self.v_mappings = nn.ModuleList([nn.Linear(d_head, d_head) for _ in range(n_heads)])

    self.d_head = d_head
    self.softmax = nn.Softmax(dim=-1)


  # understand this once? is this correct?
  def forward(self, query, key_value):
        """
        query and key_value have shape (batch_size, seq_length, token_dim)
        The token_dim must be divisible by n_heads
        """

        result = []
        for q, kv in zip(query, key_value):
            seq_result = []
            for head in range(self.n_heads):
                q_mapping = self.q_mappings[head]
                k_mapping = self.k_mappings[head]
                v_mapping = self.v_mappings[head]

                q_1 = q[:, head * self.d_head: (head + 1) * self.d_head]
                kv_1 = kv[:, head * self.d_head : (head + 1) * self.d_head]


                q1, k1, v1 = q_mapping(q_1), k_mapping(kv_1), v_mapping(kv_1)

                attention = self.softmax(q1 @ k1.T / (self.d_head ** 0.5))
                seq_result.append(attention @ v1)
            result.append(torch.hstack(seq_result))
        return torch.cat([torch.unsqueeze(r, dim=0) for r in result])

In [ ]:

class AttentionBlock(nn.Module):
  def __init__(self, hidden_d, n_heads, mlp_ratio=4, dropout=0.4):
    super(AttentionBlock, self).__init__()
    self.hidden_d = hidden_d
    self.n_heads = n_heads

    self.norm1 = nn.LayerNorm(hidden_d)
    self.mhsa = SelfAttention(hidden_d, n_heads)
    self.norm2 = nn.LayerNorm(hidden_d)
    self.mlp = nn.Sequential(
        nn.Linear(hidden_d, mlp_ratio * hidden_d),
        nn.GELU(),
        nn.Linear(mlp_ratio * hidden_d, hidden_d),
        nn.Dropout(dropout)
    )

  def forward(self, query, key_value = None):
    if key_value == None:
      key_value = query

    out = query + self.mhsa(self.norm1(query), self.norm1(key_value))
    out = out + self.mlp(self.norm2(out))
    return out

In [ ]:
class MyViT(nn.Module): # change input shape and perform transformation inside the block
  def __init__(self, device=torch.device('cpu'), nchw1=(300, 1, 300, 300), nchw2=(300, 1, 2048),
               n_patches=15, hidden_d = 256,
               n_blocks = 1, n_heads = 4, out_d = 3, dropout=0.2):
    # Super constructor
    super(MyViT, self).__init__()

    # Attributes
    self.nchw1 = nchw1 # (N, C, H, W)
    self.nchw2 = nchw2 # (N, C, D)

    self.n_patches = n_patches
    self.hidden_d = hidden_d
    self.device = device

    # part 1 ------------------------------
    assert nchw1[2] % n_patches == 0, "Input shape not entirely divisible by number of patches"
    assert nchw1[3] % n_patches == 0, "Input shape not entirely divisible by number of patches"

    self.patch_size = (nchw1[2] / n_patches, nchw1[3] / n_patches)

    # 1) Linear mapper
    self.input_d1 = int(nchw1[1] * self.patch_size[0] * self.patch_size[1])
    self.linear_mapper1 = nn.Linear(self.input_d1, self.hidden_d)

    self.input_d2 = int(nchw2[1] * nchw2[2])
    self.linear_mapper2 = nn.Linear(self.input_d2, self.hidden_d)

    # 2) Learnable classifiation token
    self.class_token1 = nn.Parameter(torch.rand(1, n_patches**2, self.hidden_d)) # adding a class across the number of images dimension
    self.class_token2 = nn.Parameter(torch.rand(1, self.nchw2[-2],self.hidden_d))

    # 3) Positional embedding
    self.pos_embed = nn.Parameter(torch.tensor(get_positional_embeddings(self.n_patches ** 2, self.hidden_d).clone().detach()))
    # self.pos_embed = nn.Parameter(torch.tensor(get_positional_embeddings(self.n_patches ** 2 + 1, self.hidden_d).clone().detach()))
    self.pos_embed.requires_grad = False

    # 4 transformer
    self.blocks = nn.ModuleList([AttentionBlock(hidden_d, n_heads) for _ in range(n_blocks)])


    # # 5 classification block
    self.mlp = nn.Sequential(
        nn.Linear(self.hidden_d, out_d),
        nn.Dropout(dropout),
        nn.Softmax(dim=-1)
        # nn.Sigmoid()
    )

    self.apply(self._init_weights)
  def _init_weights(self, module):
    if isinstance(module, nn.Linear):
        nn.init.xavier_uniform_(module.weight)
        if module.bias is not None:
            nn.init.constant_(module.bias, 0)
    elif isinstance(module, nn.LayerNorm):
        nn.init.constant_(module.bias, 0)
        nn.init.constant_(module.weight, 1.0)


  def forward(self, data1, data2):

    # part 1 -------- start
    # print(data1.shape, data2.shape)
    b1, n1, c1, h1, w1 = data1.shape
    b2, n2, c2, d2 = data2.shape

    patches = patchify(data1, self.n_patches).to(self.device)
    tokens1 = self.linear_mapper1(patches)

    tokens2 = self.linear_mapper2(data2)

    tokens2 = tokens2.view(b2, n2*c2, 512)


    # Adding classification token to the tokens

    tokens2 = torch.cat((self.class_token2.expand(b2, 1, -1), tokens2), dim=1)

    # tokens2 = torch.stack([torch.vstack((self.class_token2, tokens2[i])) for i in range(len(tokens2))])

    # print(tokens1.shape, tokens2.shape)
    # for batch in range(b):
    # Adding positional embedding
    pos_embed = self.pos_embed.repeat(b1, n1, 1, 1)
    tokens1 = tokens1 + pos_embed

    # tokens1 = torch.cat((self.class_token1.expand(b1, 1, tokens1.shape[2], 512), tokens1), dim=1)
    tokens1 = torch.stack([torch.vstack((self.class_token1, tokens1[i])) for i in range(len(tokens1))])


    # pos_embed = self.pos_embed.repeat(b2, n2+1, 1, 1)
    # tokens2 = tokens2 + pos_embed

    b_1, n_1, p_1, d_1 = tokens1.shape
    # b_2, n_2, c_2, d_2 = tokens2.shape




    for block in self.blocks:
      tokens2 = block(tokens2)

    print("self attention 2 done")
    new = []

    for i in range(b_1):
      out_ = tokens1[i]
      for block in self.blocks:
        out_ = block(out_)

      new.append(out_)

    print("self attention 1 done")
    tokens1 = torch.stack(new)
    # print(tokens1.shape)

    tokens1 = tokens1.view(b_1, n_1*p_1, d_1)




    # print(tokens1.shape, tokens2.shape)
    # cross attention


    for block in self.blocks:
      out_new = block(tokens2, tokens1)

    print('cross attention 1 done')
    # print(out_new.shape)

    # again self attention
    for block in self.blocks:
      out_new = block(out_new)

    # print(out_new.shape)
    print('self attention 3 done')

    out_new = out_new[:,0]

    return self.mlp(out_new)



In [ ]:
# model = MyViT((300, 1, 300, 300), n_patches=15, n_blocks=2, hidden_d=8, n_heads=2, out_d=3)
# model(data)
torch.cuda.empty_cache()
from torch.optim.lr_scheduler import StepLR

In [ ]:
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
#root_dir = "C:/Users/SCAAI_Vaibhv/Desktop/arya/rcnn transformer/data-20240615T041314Z-001/data"
#dataset = oct_data(root_dir)


#train_data, test_data = train_test_split(dataset, test_size=0.3, random_state=42)

all_labels = []
for _, _, label in dataset:
    all_labels.append(label)
all_labels = torch.tensor(all_labels)

dataloader = DataLoader(train_data, batch_size=1, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1, shuffle=True)

#_, _, labels = dataset
class_counts = torch.bincount(all_labels)
print(len(class_counts))
class_weights = 1. / class_counts.float()

# If you have 3 classes, for example:
class_weights = class_weights / class_weights.sum()
print(class_weights)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
#model = MyViT(device, n_patches=15, n_blocks=2, hidden_d=512, n_heads=4, out_d=3).to(device)
#model.load_state_dict(torch.load('model_5.pt'))
train_losses = []
train_accuracies = []
#
def train(dataloader, test_loader, model):



    N_EPOCHS = 100
    LR = 0.00089
    optimizer = Adam(model.parameters(), lr=LR, weight_decay=1e-5)
    scheduler = StepLR(optimizer, step_size=4, gamma=0.1)
    #criterion = nn.CrossEntropyLoss()




    for epoch in trange(N_EPOCHS, desc="Training"):
        train_loss = 0.0
        correct = 0
        total = 0
        model.train()
        for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1} in training", leave=False):
            x1, x2, y = batch
            x1, x2, y = x1.to(device), x2.to(device), y.to(device)

            optimizer.zero_grad()
            y_hat = model(x1, x2)

            print(y_hat, y)

            loss = criterion(y_hat, y)
            train_loss += loss.item()
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            _, predicted = torch.max(y_hat.data, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

        train_loss /= len(dataloader)
        train_accuracy = correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)


        scheduler.step()
        if (epoch+1)%5 == 0:
            torch.save(model.state_dict(), f"model_{epoch+1}.pt")

    torch.save(model.state_dict(), f"model_100.pt")


if __name__ == "__main__":
    train(dataloader, test_loader, model)

3
tensor([0.2564, 0.3590, 0.3846])


Epoch 1 in training:   0%|                                                                      | 0/70 [00:00<?, ?it/s]

self attention 2 done
self attention 1 done
cross attention 1 done
self attention 3 done
tensor([[0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward0>) tensor([0], device='cuda:0')



Epoch 1 in training:   1%|▉                                                             | 1/70 [00:16<18:34, 16.16s/it]

self attention 2 done
self attention 1 done
cross attention 1 done
self attention 3 done
tensor([[0., 0., 1.]], device='cuda:0', grad_fn=<SoftmaxBackward0>) tensor([2], device='cuda:0')


...

self attention 2 done
self attention 1 done
cross attention 1 done
self attention 3 done
tensor([[1.0000e+00, 1.8336e-12, 7.3231e-39]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>) tensor([0], device='cuda:0')



Epoch 4 in training:  53%|████████████████████████████████▏                            | 37/70 [09:18<08:31, 15.49s/it]

self attention 2 done
self attention 1 done
cross attention 1 done
self attention 3 done
tensor([[5.0000e-01, 5.0000e-01, 2.0060e-26]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>) tensor([1], device='cuda:0')



Training:   3%|█▉                                                               | 3/100 [1:03:14<34:04:50, 1264.85s/it]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), f"model_3.pt")
model.eval()
val_loss = 0.0
correct = 0
total = 0
with torch.no_grad():
    for inputs1, inputs2, labels in test_loader:
        inputs1, inputs2, labels = inputs1.to(device), inputs2.to(device), labels.to(device)
        outputs = model(inputs1, inputs2)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_loss /= len(test_loader)
val_accuracy = correct / total
val_losses.append(val_loss)
val_accuracies.append(val_accuracy)

print(f"Epoch {epoch + 1}/{N_EPOCHS} loss: {train_loss:.2f} , test_loss: {val_loss} , accuracy: {train_accuracy:.2f} , test_accuracy: {val_accuracy}")


IndentationError: unexpected indent (3938787756.py, line 3)

In [ ]:
# # Test loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = MyViT(device, n_patches=15, n_blocks=2, hidden_d=512, n_heads=4, out_d=3).to(device)
#model.load_state_dict(torch.load('model_100.pt'))
#criterion = CrossEntropyLoss()

with torch.no_grad():
    correct, total = 0, 0
    test_loss = 0.0
    for batch in tqdm(test_loader, desc="Testing"):
        x1, x2, y = batch
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        y_hat = model(x1, x2)
        loss = criterion(y_hat, y)
        test_loss += loss.detach().cpu().item() / len(test_loader)

        correct += torch.sum(torch.argmax(y_hat, dim=1) == y).detach().cpu().item()
        total += len(x1)
    print(f"Test loss: {test_loss:.2f}")
    print(f"Test accuracy: {correct / total * 100:.2f}%")


Testing:   0%|                                                                                  | 0/30 [00:00<?, ?it/s]

self attention 2 done


Testing:   3%|██▍                                                                       | 1/30 [00:12<06:08, 12.71s/it]

self attention 1 done
cross attention 1 done
self attention 3 done
self attention 2 done


Testing:   7%|████▉                                                                     | 2/30 [00:24<05:37, 12.05s/it]

...

self attention 1 done
cross attention 1 done
self attention 3 done
self attention 2 done


Testing:  97%|██████████████████████████████████████████████████████████████████████▌  | 29/30 [05:47<00:12, 12.44s/it]

self attention 1 done
cross attention 1 done
self attention 3 done
self attention 2 done


Testing: 100%|█████████████████████████████████████████████████████████████████████████| 30/30 [05:59<00:00, 11.99s/it]

self attention 1 done
cross attention 1 done
self attention 3 done
Test loss: 1.06
Test accuracy: 46.67%


In [ ]:
# Initialize variables for TP, FP, TN, FN for each class
TP = [0, 0, 0]
FP = [0, 0, 0]
TN = [0, 0, 0]
FN = [0, 0, 0]

# Iterate through the test set again
outputs = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Calculating metrics"):
        x1, x2, y = batch
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        y_hat = model(x1, x2)
        outputs.append(y_hat)

        # Calculate predictions
        predictions = torch.argmax(y_hat, dim=1)

        # Update TP, FP, TN, FN counts for each class
        for i in range(3):
            TP[i] += ((predictions == i) & (y == i)).sum().item()
            FP[i] += ((predictions == i) & (y != i)).sum().item()
            TN[i] += ((predictions != i) & (y != i)).sum().item()
            FN[i] += ((predictions != i) & (y == i)).sum().item()

# Calculate metrics for each class
precision = []
recall = []
sensitivity = []
specificity = []
f1_scores = []

# Calculate F1 score for each class


for i in range(3):
    precision.append(TP[i] / (TP[i] + FP[i]))
    recall.append(TP[i] / (TP[i] + FN[i]))
    sensitivity.append(recall[i])  # sensitivity is the same as recall
    specificity.append(TN[i] / (TN[i] + FP[i]))

for i in range(3):
    if precision[i] + recall[i] == 0:
        f1_scores.append(0.0)
    else:
        f1_scores.append(2 * (precision[i] * recall[i]) / (precision[i] + recall[i]))
# Print metrics for each class
for i in range(3):
    print(f"Class {i}:")
    print(f"  Precision: {precision[i]:.2f}")
    print(f"  Recall: {recall[i]:.2f}")
    print(f"  Sensitivity: {sensitivity[i]:.2f}")
    print(f"  Specificity: {specificity[i]:.2f}")
    print(f"  F1-score: {f1_scores[i]:.2f}")


Calculating metrics:   0%|                                                                      | 0/30 [00:00<?, ?it/s]

self attention 2 done


Calculating metrics:   3%|██                                                            | 1/30 [00:12<06:14, 12.90s/it]

self attention 1 done
cross attention 1 done
self attention 3 done
self attention 2 done


Calculating metrics:   7%|████▏                                                         | 2/30 [00:25<06:01, 12.92s/it]

...

Calculating metrics:  97%|██████████████████████████████████████████████████████████▉  | 29/30 [05:34<00:11, 11.94s/it]

self attention 1 done
cross attention 1 done
self attention 3 done
self attention 2 done


Calculating metrics: 100%|█████████████████████████████████████████████████████████████| 30/30 [05:45<00:00, 11.52s/it]

self attention 1 done
cross attention 1 done
self attention 3 done


ZeroDivisionError: division by zero

In [ ]:
# def train(dataloader, test_loader):


#     # Defining model and training options
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     model = MyViT(device, n_patches=15, n_blocks=1, hidden_d=512, n_heads=2, out_d=3).to(device)
#     # model.load_state_dict(torch.load('model_100.pt'))
#     N_EPOCHS = 100
#     LR = 0.001

#     # Training loop
#         # Training loop
#     optimizer = Adam(model.parameters(), lr=LR)

#     scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

#     criterion = CrossEntropyLoss()
#     # criterion = nn.BCELoss()



#     train_losses = []


#     for epoch in trange(N_EPOCHS, desc="Training"):
#         train_loss = 0.0
#         for batch in tqdm(
#             dataloader, desc=f"Epoch {epoch + 1} in training", leave=False
#         ):
#             x1, x2, y = batch
#             x1, x2, y = x1.to(device), x2.to(device), y.to(device)  # Move tensors to GPU
#             # for i i

#             optimizer.zero_grad()

#             y_hat = model(x1, x2)

#             # y_one_hot = F.one_hot(y, num_classes=3).float()

#             print(y_hat, y)

#             # return y_hat
#             loss = criterion(y_hat, y)

#             train_loss += loss.detach().cpu().item() / len(dataloader)

#             # _, predicted = torch.max(y_hat.data, 1)


#             loss.backward()
#             optimizer.step()

#         print(f"Epoch {epoch + 1}/{N_EPOCHS} loss: {train_loss:.2f}")
#         train_losses.append(train_loss)

#         scheduler.step()
#         if (epoch + 1) % 10 == 0:
#             torch.save(model.state_dict(), f"model_10.pt")

#         model.eval()
#         val_loss = 0
#         correct = 0
#         total = 0
#         with torch.no_grad():
#             for inputs1, inputs2, labels in test_loader:
#                 inputs1, inputs2, labels = inputs1.to(device), inputs2.to(device), labels.to(device)
#                 outputs = model(inputs1, inputs2)
#                 loss = criterion(outputs, labels)
#                 val_loss += loss.item()
#                 _, predicted = torch.max(outputs, 1)
#                 total += labels.size(0)
#                 correct += (predicted == labels).sum().item()

In [ ]:
outputs

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Inside your main function or after the test loop
# Assuming you have already defined the test_dataloader and the model is trained

model.eval()  # Set the model to evaluation mode
y_true = []
y_pred = []

with torch.no_grad():
    for batch in tqdm(test_dataloader, desc="Testing"):
        x1, x2, y = batch
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        y_hat = model(x1, x2)
        predicted_labels = torch.argmax(y_hat, dim=1)

        y_true.extend(y.cpu().numpy())
        y_pred.extend(predicted_labels.cpu().numpy())

# Calculate precision and recall
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')

print(f'Precision: {precision}')
print(f'Recall: {recall}')

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Fine tuning resnet 50 on the dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# dataloader with features also output
import os
import torchvision.transforms as transforms
from torchvision.models import resnet50
# importing libraries
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm import trange, tqdm
from torchvision import models, transforms
import torchvision.transforms.functional as TF


class oct_data(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data = []
        self.labels = []

        classes = sorted(os.listdir(root_dir))

        for class_idx, folder in enumerate(classes):
            class_dir = os.path.join(root_dir, folder)
            for p in os.listdir(class_dir):
                path = os.path.join(class_dir, p)
                self.data.append(path)
                self.labels.append(class_idx)
                # self.max = max(len(os.listdir(path)), self.max)

        # Preprocess transforms for images
        self.preprocess = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        # Precompute and cache features during initialization

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        label = self.labels[idx]
        original_images = []
        data_path = self.data[idx]

        for path in os.listdir(data_path):
            image_path = os.path.join(data_path, path)
            img = Image.open(image_path).convert('RGB')
            img = self.preprocess(img)
            original_images.append(img)

        # original_images = [transforms.ToTensor()(img) for img in original_images]



        return original_images, label


In [ ]:
dataset = oct_data('/content/drive/MyDrive/datasets/OCT/data')

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

t1 = DataLoader(train_data, batch_size=4, shuffle=True, num_workers=4)
t2 = DataLoader(test_data, batch_size=4, shuffle=True, num_workers=4)

In [ ]:
num_classes = 3

model = resnet50(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, num_classes)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
# Define the number of epochs
num_epochs = 10

# Train the model
for epoch in range(num_epochs):
    # Train the model on the training set
    model.train()
    train_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # Move the data to the device
        # inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        batch_outputs = []
        for batch in inputs:
          for images in batch:
            images = torch.stack(images).to(device)  # Stack list of images into a tensor
            outputs = model(images)
            # Aggregate outputs, e.g., by averaging
            # aggregated_output = outputs.mean(dim=0)
            # batch_outputs.append(aggregated_output)
            loss = criterion(outputs, labels)
            loss.backward()


        # batch_outputs = torch.stack(batch_outputs)

        # loss = criterion(batch_outputs, labels)
        # loss.backward()
        optimizer.step()

        # Update the training loss
        train_loss += loss.item() * inputs.size(0)

    # Evaluate the model on the test set
    model.eval()
    test_loss = 0.0
    test_acc = 0.0
    with torch.no_grad():
        for inputs_list, labels in test_loader:
            labels = labels.to(device)
            batch_outputs = []
            for images in inputs_list:
                images = torch.stack(images).to(device)
                outputs = model(images)
                aggregated_output = outputs.mean(dim=0)
                batch_outputs.append(aggregated_output)

            batch_outputs = torch.stack(batch_outputs)
            loss = criterion(batch_outputs, labels)
            test_loss += loss.item() * len(labels)
            _, preds = torch.max(batch_outputs, 1)
            test_acc += torch.sum(preds == labels.data)

    train_loss /= len(train_data)
    test_loss /= len(test_data)
    test_acc = test_acc.double() / len(test_data)
    print(f"Epoch [{epoch + 1}/{num_epochs}] Train Loss: {train_loss:.4f} Test Loss: {test_loss:.4f} Test Acc: {test_acc:.4f}")